### 피봇테이블
- 피봇테이블(pivot table) : 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


- 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름
- 데이터가 존재하지 않으면 해당 칸에 NaN 값을 넣는다.

In [3]:
df1.pivot('도시','연도','인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [4]:
df1.set_index(['도시','연도'])[['인구']].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

- 행 인덱스나 열 인덱스를 리스트로 주는 경우에는 다중 인덱스 피봇 테이블을 생성한다.

In [5]:
df1.pivot(['지역','도시'],'연도','인구')

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

- 행 인덱스와 열 인덱스는 데이터를 찾는 키(key)의 역할을 한다. 따라서 키 값으로 데이터가 단 하나만 찾아져야 한다. 만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다.

In [6]:
try:
    df1.pivot('지역','연도','인구')
except ValueError as e:
    print('ValueError:',e)

ValueError: Index contains duplicate entries, cannot reshape


### 그룹분석
- 만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석(group analysis)을 해야 한다.
- 그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산한다.

#### groupby 메서드
- 데이터를 그룹 별로 분류하는 역할을 한다.

### 그룹연산 메서드
- size, count : 그룹 데이터의 갯수
- mean, median, min, max : 그룹 데이터의 평균, 중앙값, 최소, 최대
- sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
- first, last : 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터
- agg, aggregate : 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다. 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.
- describe : 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
- apply : describe처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
- transform : 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

In [7]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1' : ['A','A','B','B','A'],
    'key2' : ['one','two','one','two','one'],
    'data1' : [1,2,3,4,5],
    'data2' : [10,20,30,40,50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


- key1의 값(A 또는 B)에 따른 data1의 평균 구하기

In [8]:
groups = df2.groupby(df2.key1)
groups

In [9]:
groups.groups # Groupby 클래스 객체에는 각 그룹 데이터의 인덱스를 저장한 groups 속성이 있다.

{'A': [0, 1, 4], 'B': [2, 3]}

In [10]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [11]:
df2.data1.groupby(df2.key1).sum() # Groupby 클래스 객체를 명시적으로 얻을 필요가 없을 때

key1
A    8
B    7
Name: data1, dtype: int64

In [14]:
df2.groupby(df2.key1)['data1'] .sum() # Groupby 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [15]:
df2.groupby(df2.key1).sum()['data1'] # 전체 데이터를 분석한 후 data1만 선택하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [17]:
df2.data1.groupby([df2.key1, df2.key2]).sum() # 키가 복수이면 리스트를 사용

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [18]:
df2.data1.groupby([df2['key1'],df2['key2']]).sum().unstack('key2')

key2,one,two
key1,,
A,6,2
B,3,4


In [25]:
df1['인구'].groupby([df1['지역'],df1['연도']]).sum().unstack('연도')

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


In [26]:
import seaborn as sns
iris = sns.load_dataset('iris')

- 각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율 구하기

In [27]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [28]:
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

In [29]:
def top3_petal_length(df):
    return df.sort_values(by='petal_length', ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length) # 데이터프레임을 만듬

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [30]:
def q3cut(s):
    return pd.qcut(s,3,labels=['소','중','대']).astype(str)

iris['petal_length_class'] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[['petal_length','petal_length_class']].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소
